In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Végtelen egyensúlyi pont 3

Ez az interaktív példa bemutat egy $2\times2$-es rendszert, aminek végtelen sok egyensúlyi pontja van az $x_1=-x_2$ egyenesen (az elméleti háttérhez nézze át az [Egyensúlyi pontok](SS-13-Egyensúlyi_pontok.ipynb) példát).

Ahhoz, hogy a $x_1=-x_2$ egyenesen helyezkedjenek el az egyensúlyi pontok, szükséges, hogy:
$$
A\bar{x}=0 \quad \forall \, \bar{x}\in\begin{bmatrix} \alpha \\ -\alpha\end{bmatrix} \, \text{és} \, \alpha\in\mathbb{R},
$$
azaz, $\begin{bmatrix} \alpha \\ -\alpha\end{bmatrix}$-nak $A$ mátrix nullteréhez kell tartoznia.

### Hogyan működik a példa?
- Módosítsa $A$ mátrixot és figyelje meg hogyan változnak az egyensúlyi pontok.
- Próbálja meg elérni $A$ mátrix módosításával, hogy az egyensúlyi pontok az kívánt egyenesre essenek.

In [2]:
#Preparatory Cell 

import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt
import sympy as sym

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#create a NxM matrix widget 
def createMatrixWidget(n,m):
    M = widgets.GridBox(children=[widgets.FloatText(layout=widgets.Layout(width='100px', height='40px'),
    value=0.0, disabled=False, label=i) for i in range(n*m)],
    layout=widgets.Layout(
        #width='50%',
        grid_template_columns= ''.join(['100px ' for i in range(m)]),
        #grid_template_rows='80px 80px 80px',
        grid_row_gap='0px',
        track_size='0px')
    )
    return M


#extract matrix from widgets and convert to numpy matrix
def getNumpyMatFromWidget(M,n,m):
    #get W gridbox dims
    M_ = numpy.matrix(numpy.zeros((n,m)))
    for irow in range(0,n):
        for icol in range(0,m):
            M_[irow,icol] = M.children[irow*3+icol].value

            
#this is a simple derived class from FloatText used to experience with interact             
class floatWidget(widgets.FloatText):
    def __init__(self,**kwargs):
        #self.n = n
        self.value = 30.0
        #self.M = 
        widgets.FloatText.__init__(self, **kwargs)

#    def value(self):
#        return 0 #self.FloatText.value

from traitlets import Unicode
from ipywidgets import register 


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

        

        
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define the matrixes
A=matrixWidget(2,2)
A.setM(numpy.matrix('1. 0.; 0. 1.'))

def main_callback(matA,DW):
    
    As = sym.Matrix(matA)
    NAs = As.nullspace()
    
    t = numpy.linspace(-10,10,1000)
    if len(NAs) == 1:
        eq1 = [t[i]*numpy.matrix(NAs[0]) for i in range(0,len(t))]
        x1 = [eq1[i][0,0] for i in range(0,len(t))]
        x2 = [eq1[i][1,0] for i in range(0,len(t))]
    
    fig = plt.figure(figsize=(6,6))
    if len(NAs) == 0:
        plt.plot(0,0,'bo')
    if len(NAs) == 1:
        plt.plot(x1,x2)
    if len(NAs) == 2:
        plt.fill((-5,-5,5,5),(-5,5,5,-5),alpha=0.5)
        plt.xlim(left=-5,right=5)
        plt.ylim(top=5,bottom=-5)
    plt.grid()
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    print('A nullterének bázisa (sorok szerint) %s. \nA sajátértékek %s' %(str(numpy.array(NAs)),
                                                                          str(numpy.linalg.eig(matA)[0])))


#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Vizsgálat',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Vizsgálat',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

out = widgets.interactive_output(main_callback,{'matA':A,'DW':DW})
out1 = widgets.HBox([out,
                     widgets.VBox([widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label('$\qquad \qquad A=$')]),
                     widgets.VBox([widgets.Label(''),widgets.Label(''),widgets.Label(''),A,START])])
out.layout.height = '450px'
display(out1)

In [4]:
#create dummy widget 2
DW2 = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))
DW2.value = -1

#create button widget
START2 = widgets.Button(
    description='Válaszok felfedése',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Kattintson a válaszok felfedéséhez',
    icon='check'
)
                       
def on_start_button_clicked2(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW2.value> 0 :
        DW2.value = -1
    else: 
        DW2.value = 1
    pass
START2.on_click(on_start_button_clicked2)

def main_callback2(DW2):
    if DW2 > 0:
        display(Markdown(r'''Válasz: 
        A mátrix létrehozásához olyan sorvektorok választása szükséges, amik merőlegesek a nulltérre. Egy lehetséges mátrix például:
$$
A=\begin{bmatrix} 1 & 1 \\ 2 & 2 \end{bmatrix}.
$$'''))
    else:
        display(Markdown(''))

#create a graphic structure to hold all widgets 
alltogether2 =  widgets.VBox([START2])

out2 = widgets.interactive_output(main_callback2,{'DW2':DW2})
#out.layout.height = '300px'
display(out2,alltogether2)

Output()